## AI-HUB 데이터 우선 정제, 중립감정 활용할 데이터 있음. & 2차분류 데이터 미리 만들기
https://sig413.tistory.com/5 AI-HUB 예전 데이터 공유해달라 댓글씀.

In [1]:
import pandas as pd
import plotly.express as px

In [16]:
df1 = pd.read_csv("../감성대화/감성대화말뭉치(최종데이터)_Training.csv", encoding='UTF-8')
df2 = pd.read_csv("../감성대화/감성대화말뭉치(최종데이터)_Validation.csv", encoding='UTF-8')

In [17]:
df1 = df1[['감정_대분류', '사람문장1']] # 감성대화중 감정 대분류에 해당하는 것은 '사람문장1'이기 때문에 '감정대분류'와 '사람문장1'만 남겨놓음.
df1.rename(columns={"감정_대분류": "sentiment", "사람문장1":"sentence"}, inplace=True) # column명 통일
df1 = df1[["sentence", "sentiment"]] # column순서 변경

# df2에 대해서도 동일한 과정 실시
df2 = df2[['감정_대분류', '사람문장1']]
df2.rename(columns={"감정_대분류": "sentiment", "사람문장1":"sentence"}, inplace=True)
df2 = df2[["sentence", "sentiment"]]

In [18]:
result = pd.concat([df1,df2])
result.drop_duplicates(inplace=True)
result.reset_index(drop=True, inplace=True)
result['sentiment'] = result['sentiment'].apply(lambda x: "분노" if x == "상처" else x) # 상처와 분노를 동종의 감정으로 간주.
result['sentiment'] = result['sentiment'].apply(lambda x: "불안" if x == "당황" else x) # 당황과 불안을 동종의 감정으로 간주.
result.to_csv("./AIhub_data1.csv", index=False, encoding='UTF-8')
result

,sentence,sentiment
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,분노
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,분노
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,분노
...,...,...
58251,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...,불안
58252,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아.,불안
58253,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.,불안
58254,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...,불안


In [19]:
result.isnull().sum()

sentence     0
sentiment    0
dtype: int64

In [22]:
result['sentiment'].unique()

array(['분노', '기쁨', '불안', '슬픔'], dtype=object)

In [23]:
# "sentiment" 컬럼의 값들의 분포를 막대그래프로 그리기
fig = px.histogram(result, x='sentiment', title='Sentiment Distribution')

# fig의 크기 조절 (width와 height를 원하는 크기로 설정합니다.)
fig.update_layout(
    width=800,  # 가로 크기
    height=500,  # 세로 크기
    xaxis=dict(tickfont=dict(size=14, family='Arial, bold')),  # x축 레이블 폰트 크기와 스타일 조절
    yaxis=dict(tickfont=dict(size=14, family='Arial, bold')),  # y축 레이블 폰트 크기와 스타일 조절
)

# 색상을 랜덤으로 지정
# 'color_discrete_sequence' 파라미터에 'px.colors.qualitative'를 사용하면 Plotly에서 제공하는 랜덤 색상을 사용할 수 있습니다.
fig.update_traces(marker=dict(color=px.colors.qualitative.Prism))

# 막대그래프의 모양 변경 (width 속성을 여기서 조절)
fig.update_traces(
    marker=dict(color=px.colors.qualitative.Prism),  # 색상을 랜덤하게 지정
    marker_line_width=3,  # 테두리 선 두께 조절
    marker_line_color='black',  # 테두리 선 색상 조절
    opacity=0.9,  # 막대의 투명도 조절 (0부터 1 사이의 값을 사용, 1은 완전 불투명)
)

# 그래프 출력
fig.show()

In [24]:
sentiment_distribution = result['sentiment'].value_counts()

print(sentiment_distribution)

분노    20562
불안    20231
슬픔    10125
기쁨     7338
Name: sentiment, dtype: int64


In [25]:
df3 = pd.read_csv("../감정 분류를 위한 대화 음성 데이터셋/4차년도.csv", encoding='cp949')
df3.rename(columns={'4번감정세기':'4번 감정세기'}, inplace=True)

df4 = pd.read_csv("../감정 분류를 위한 대화 음성 데이터셋/5차년도.csv", encoding='cp949')
df4.rename(columns={'4번감정세기':'4번 감정세기'}, inplace=True)

df5 = pd.read_csv("../감정 분류를 위한 대화 음성 데이터셋/5차년도_2차.csv", encoding='cp949')
df5.rename(columns={'4번감정세기':'4번 감정세기'}, inplace=True)

In [26]:
def emo_sum(dataframe): # 각 row의 감정세기를 계산하는 함수
    rep_emo = []
    for index, row in dataframe.iterrows():
        emo = {}
        for i in dataframe.columns[3:13:2]:
            emo[row[i]] = emo.get(row[i], 0) + row[i+"세기"]
        # 딕셔너리의 값들 중에서 최댓값 찾기
        max_value = max(emo.values())

        # 최댓값에 해당하는 key 찾기
        max_key_list = [key for key, value in emo.items() if value == max_value]
        if len(max_key_list) > 1:
            rep_emo.append(None)
        else:
            rep_emo.append(max_key_list[0])
    return rep_emo

In [27]:
df3["sentiment"] = emo_sum(df3) # 함수를 통해 출력된 감정 리스트를 데이터프레임에 추가
df3 = df3[['발화문', 'sentiment']] # 문장과 감정만 남겨놓음
df3.dropna(inplace=True)
df3.rename(columns={'발화문': 'sentence'}, inplace=True) # column명 통일

df4["sentiment"] = emo_sum(df4) # 함수를 통해 출력된 감정 리스트를 데이터프레임에 추가
df4 = df4[['발화문', 'sentiment']] # 문장과 감정만 남겨놓음
df4.dropna(inplace=True)
df4.rename(columns={'발화문': 'sentence'}, inplace=True) # column명 통일

df5["sentiment"] = emo_sum(df5) # 함수를 통해 출력된 감정 리스트를 데이터프레임에 추가
df5 = df5[['발화문', 'sentiment']] # 문장과 감정만 남겨놓음
df5.rename(columns={'발화문': 'sentence'}, inplace=True) # column명 통일
df5.dropna(inplace=True)
df5['sentiment'] = df5['sentiment'].apply(lambda x : x.capitalize())

In [32]:
result2 = pd.concat([df3,df4,df5])
result2.drop_duplicates(inplace=True, keep='last')
result2.reset_index(drop=True, inplace=True)
result2 = result2[result2['sentiment'] != "Neutral"] # 중립(Neutral)의 감정은 이미 Primary classification에서 사용했으므로 제외.
result2['sentiment'] = result2['sentiment'].apply(lambda x: "분노" if x == "Fear" or x == "Disgust" else x) # Fear 와 Digust를 분노의 감정으로 간주
result2['sentiment'] = result2['sentiment'].apply(lambda x: "열정" if x == "Surprise" else x) # Surprise를 열정의 감정으로 간주
result2['sentiment'] = result2['sentiment'].apply(lambda x: "슬픔" if x == "Sadness" else x) # 한글로 통일
result2['sentiment'] = result2['sentiment'].apply(lambda x: "기쁨" if x == "Happiness" else x) # 한글로 통일
result2['sentiment'] = result2['sentiment'].apply(lambda x: "분노" if x == "Angry" else x) # 한글로 통일
result2.to_csv("./AIhub_data2.csv", index=False, encoding='UTF-8')
result2

,sentence,sentiment
0,"어, 청소 니가 대신 해 줘!",분노
1,둘 다 청소 하기 싫어. 귀찮아.,분노
2,둘 다 하기 싫어서 화내.,분노
3,그럼 방세는 어떡해.,슬픔
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,슬픔
...,...,...
33958,맞아. 나한테만 퉁명스럽고 일을 많이 시키더라고.,슬픔
33959,나 엘리베이터에 갇혔어.,슬픔
33960,하지만 기분이 나쁜 걸 어떡해?,슬픔
33961,자취방 엘리베이턴데 정전인가봐.,슬픔


In [33]:
result2.isnull().sum()

sentence     0
sentiment    0
dtype: int64

In [34]:
result2['sentiment'].unique()

array(['분노', '슬픔', '기쁨', '열정'], dtype=object)

In [35]:
# "sentiment" 컬럼의 값들의 분포를 막대그래프로 그리기
fig = px.histogram(result2, x='sentiment', title='Sentiment Distribution')

# fig의 크기 조절 (width와 height를 원하는 크기로 설정합니다.)
fig.update_layout(
    width=800,  # 가로 크기
    height=500,  # 세로 크기
    xaxis=dict(tickfont=dict(size=14, family='Arial, bold')),  # x축 레이블 폰트 크기와 스타일 조절
    yaxis=dict(tickfont=dict(size=14, family='Arial, bold')),  # y축 레이블 폰트 크기와 스타일 조절
)

# 색상을 랜덤으로 지정
# 'color_discrete_sequence' 파라미터에 'px.colors.qualitative'를 사용하면 Plotly에서 제공하는 랜덤 색상을 사용할 수 있습니다.
fig.update_traces(marker=dict(color=px.colors.qualitative.Prism))

# 막대그래프의 모양 변경 (width 속성을 여기서 조절)
fig.update_traces(
    marker=dict(color=px.colors.qualitative.Prism),  # 색상을 랜덤하게 지정
    marker_line_width=3,  # 테두리 선 두께 조절
    marker_line_color='black',  # 테두리 선 색상 조절
    opacity=0.9,  # 막대의 투명도 조절 (0부터 1 사이의 값을 사용, 1은 완전 불투명)
)

# 그래프 출력
fig.show()

In [36]:
sentiment_distribution = result2['sentiment'].value_counts()
print(sentiment_distribution)

슬픔    16309
분노    11932
기쁨     4016
열정     1173
Name: sentiment, dtype: int64


In [ ]:
# data6 = pd.read_csv("./chatbot_dataset.csv", encoding='UTF-8')
# data6 = data6[['Q', 'label']] # 활용할 columns만 남겨놓음
# data6.rename(columns={'Q': 'sentence', "label":"sentiment"}, inplace=True) # column명 통일
# data6
#data6.to_csv("사랑이별_커뮤니티_챗봇데이터.csv", index=False, encoding='UTF-8') # 긍부정으로 나눠서 긍정이면 사랑, 부정이면 이별로 구별할 것.

## 7/26 오늘 할거 다준비했다있ㅆㅆㅆㅆㅆ

In [ ]:
data6 = pd.read_csv("사랑이별_커뮤니티_챗봇데이터_전처리완.csv", encoding='UTF-8')

In [37]:
sec_clas = pd.concat([result,result2])
print(sec_clas['sentiment'].value_counts())
sec_clas

분노    32494
슬픔    26434
불안    20231
기쁨    11354
열정     1173
Name: sentiment, dtype: int64


,sentence,sentiment
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,분노
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,분노
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,분노
...,...,...
33958,맞아. 나한테만 퉁명스럽고 일을 많이 시키더라고.,슬픔
33959,나 엘리베이터에 갇혔어.,슬픔
33960,하지만 기분이 나쁜 걸 어떡해?,슬픔
33961,자취방 엘리베이턴데 정전인가봐.,슬픔


## 긍정/부정/중립 1차 감정분류를 위한 학습데이터 전처리과정.

In [2]:
import pandas as pd

In [3]:
# 긍부정 라벨링된 데이터셋 정규화 과정
data1 = pd.read_csv("./multi_sentiment_dataset.csv", encoding='UTF-8') # AI hub에 있는 다감정 대화데이터셋인데 https://sig413.tistory.com/5 여기서 그냥 긍부정으로 나눠버림...
data1.rename(columns={'발화': 'sentence', "감정_int":"sentiment"}, inplace=True) # column명 통일

data2 = pd.read_csv("./ratings.txt", sep="\t", encoding='UTF-8') # 긍부정으로 라벨링된 네이버 영화리뷰 데이터셋 
data2 = data2[['document', 'label']] # 문장과 감정만 남겨놓음
data2.rename(columns={'document': 'sentence', "label":"sentiment"}, inplace=True) # column명 통일

data3 = pd.read_csv("./naver_shopping.txt", sep="\t", encoding='UTF-8') # 긍부정으로 라벨링된 네이버 쇼핑리뷰 데이터셋 
data3 = data3[['sentence', 'sentiment']] # columns 순서 변경
data3['sentiment'] = data3['sentiment'].apply(lambda x: 1 if x >= 4 else 0) # 'sentiment' column 값 변경 4~5점을 1(긍정)으로 1~2점을 0(부정)으로 변환

data4 = pd.read_csv("./steam.txt", sep="\t", encoding='UTF-8') # 긍부정으로 라벨링된 스팀 한국어 리뷰 데이터셋
data4 = data4[['sentence', 'sentiment']] # columns 순서 변경

data5 = pd.read_csv("./chatbot_dataset.csv", encoding='UTF-8') # 챗봇 트레이닝용 문답 데이터셋, 답어가 중립감정이라 생각하여 활용할 예정. 나머지는 다감정분석에 활용할 것.
data5 = data5[['A', 'label']] # 활용할 columns만 남겨놓음
data5.rename(columns={'A': 'sentence', "label":"sentiment"}, inplace=True) # column명 통일
data5['sentiment'] = data5['sentiment'].apply(lambda x: 2) # 중립인 감정의 'sentiment' 값을 2로 함

data6 = pd.read_csv("./AIhub_data2.csv", encoding='UTF-8') # AI hub에 있는 다감정 대화데이터셋
data6 = data6[data6["sentiment"] == "Neutral"]
data6['sentiment'] = data6['sentiment'].apply(lambda x: 2) # 중립인 감정의 'sentiment' 값을 2로 함

result3 = pd.concat([data1,data2,data3,data4,data5,data6])

In [5]:
result3['sentiment'] = result3['sentiment'].apply(lambda x: "부정" if x == 0 else "중립" if x == 2 else "긍정")

In [6]:
result3['sentiment'].value_counts()

부정    257108
긍정    257026
중립     12357
Name: sentiment, dtype: int64

In [8]:
result3.isnull().sum()

sentence     8
sentiment    0
dtype: int64

In [9]:
result3.dropna(axis=0, inplace=True)
result3.reset_index(drop=True, inplace=True)
result3.to_csv("./primary_classification_data.csv", index=False, encoding='UTF-8')